<a href="https://colab.research.google.com/github/anushk4/4_color/blob/main/Another_copy_of_Seamless_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Seamless Tutorial


## Quick Links

1. seamless_communication GitHub repository: https://github.com/facebookresearch/seamless_communication
2. fairseq2 Github repository: https://github.com/facebookresearch/fairseq2
3. HuggingFace: https://huggingface.co/collections/facebook/seamless-communication-6568d486ef451c6ba62c7724
4. Seamless demos: https://seamless.metademolab.com/
5. Fleurs datasets for evaluation: https://huggingface.co/datasets/google/fleurs/tree/main/data

### Set up seamless_communication, fairseq2 and some utilities.

In [ ]:
%%capture
!pip install fairseq2
!pip install pydub sentencepiece
!pip install git+https://github.com/facebookresearch/seamless_communication.git

In [ ]:
import io
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import mmap
import numpy
import soundfile
import torchaudio
import torch

from collections import defaultdict
from IPython.display import Audio, display
from pathlib import Path
from pydub import AudioSegment

from seamless_communication.inference import Translator
from seamless_communication.streaming.dataloaders.s2tt import SileroVADSilenceRemover

# SeamlessM4T Inference:

## Initialize the models:

In [ ]:
# Initialize a Translator object with a multitask model, vocoder on the GPU.

model_name = "seamlessM4T_v2_large"
vocoder_name = "vocoder_v2" if model_name == "seamlessM4T_v2_large" else "vocoder_36langs"

translator = Translator(
    model_name,
    vocoder_name,
    device=torch.device("cuda:0"),
    dtype=torch.float16,
)

100%|██████████| 8.45G/8.45G [00:53<00:00, 171MB/s]
100%|██████████| 360k/360k [00:00<00:00, 10.2MB/s]
100%|██████████| 4.93M/4.93M [00:00<00:00, 62.0MB/s]
Using the cached tokenizer of seamlessM4T_v2_large. Set `force` to `True` to download again.
100%|██████████| 160M/160M [00:00<00:00, 197MB/s]
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


## S2ST inference:

In [ ]:
# README:  https://github.com/facebookresearch/seamless_communication/tree/main/src/seamless_communication/cli/m4t/predict
# Please use audios with duration under 20 seconds for optimal performance.

# Resample the audio in 16khz if sample rate is not 16khz already.
# torchaudio.functional.resample(audio, orig_freq=orig_freq, new_freq=16_000)

print("English audio:")
in_file = "/content/sudio-new1_7OXaXBJI.mp3"
display(Audio(in_file, rate=16000, autoplay=False, normalize=True))

tgt_langs = "hin"

text_output, speech_output = translator.predict(
    input=in_file,
    task_str="S2ST",
    tgt_lang=tgt_langs
)

  # print(f"Translated text in {tgt_langs}: {text_output}")
  # print()

out_file = f"/content/translated_LJ_{tgt_langs}.wav"

torchaudio.save(out_file, speech_output.audio_wavs[0][0].to(torch.float32).cpu(), speech_output.sample_rate)
audio_play = Audio(out_file, rate=speech_output.sample_rate, autoplay=False, normalize=True)
display(audio_play)
print()

English audio:
